In [1]:
## Apriori algorithm
import pandas as pd
from pandas import DataFrame
import itertools

In [2]:
def conv(val):
        if val == 't':
            return 1
        else:
            return 0
        
class CSVLoader():
    def __init__(self) -> None:
        pass
    # load csv

    def load_csv(self, csv_name: str) -> DataFrame:
        self.column_list = pd.read_csv(csv_name, index_col=0, nrows=0).columns.tolist()
        csv_header_len = len(self.column_list) + 1
        csv_file = pd.read_csv(csv_name, engine="python", converters={k: conv for k in range(csv_header_len)})
        return csv_file

In [44]:
class Apriori():

    def __init__(self, support: float, confidence: float, csv_name: str) -> None:
        csv_loader = CSVLoader()
        self.data = csv_loader.load_csv(csv_name=csv_name)
        self.min_support = support
        self.confidence = confidence
        self.row_count = len(self.data.index)

    def calculate_support(self, data: DataFrame, combinations: list = None) -> dict:
        if combinations:
            support_count = {}
            count = 0
            for item in combinations:
                for row, row_data in self.data.iterrows():
                    add_frequency = True
                    for member in item:
                        if not row_data[member]:
                            add_frequency = False
                            break
                    if add_frequency:
                        count += 1
                print(len(support_count))
                support_count[item] = count/self.row_count
                count = 0
        else:
            support_count = {item:0 for item in data.columns}
            for column in data.columns:
                support_count[column] = data[column].value_counts()[1]/self.row_count
        return support_count

    def prune(self, current_combinations: list, previous_frequent_combinations: list, previous_length: int) -> list:
        # remove all items from set that are not part of previous frequent list
        # combinations = [(1,2,3), ... , (1,2,3)]
        new_combinations_list = current_combinations
        previous_frequent_combinations = set(previous_frequent_combinations)
        for index, item in enumerate(current_combinations):
            temp_combinations = list(itertools.combinations(item, previous_length))
            for prev_item in temp_combinations:
                temp_set = set(prev_item)
                if not temp_set.issubset(previous_frequent_combinations):
                    new_combinations_list.pop(index)
                    break
        return new_combinations_list

    def above_min_support(self, frequency_dict: dict) -> list:
        above_min_support = []
        for key in frequency_dict.keys():
            if frequency_dict[key] > self.min_support:
                above_min_support.append(key)
        return above_min_support

    def apriori(self) -> list:
        #Run apriori
        #read file and format
        c1 = self.calculate_support(self.data)
        l1 = self.above_min_support(c1)
        # print(l1)

        accumulated_frequent_dataset = {}

        current_lset = l1
        current_iteration = 2
        test = True
        while current_lset and test:
            #store frequent item set TODO, add support values to this accumulated to not calculate again
            accumulated_frequent_dataset[current_iteration - 1] = current_lset
            print(len(current_lset))
            # create combinations
            combinations = list(itertools.combinations(current_lset, current_iteration))
            print(len(combinations))
            # prune
            combinations = self.prune(current_combinations=combinations, previous_frequent_combinations=current_lset, previous_length=current_iteration-1)
            # calculate above min support
            support_values = self.calculate_support(self.data, combinations=combinations)
            current_lset = self.above_min_support(support_values)
            print(current_lset)
            print(len(current_lset))
            # current lset from here
            current_iteration += 1
            # print(len(combinations))
            # print(accumulated_frequent_dataset)
            test = False
        
        # add association rules TODO: create function
        return []# list of association rules

def main():
    support = 0.005
    confidence = 0.6
    csv_name = "./myDataFile.csv"
    apriori = Apriori(support=support, confidence=confidence, csv_name=csv_name)
    association_rules = apriori.apriori()
    # print(association_rules)

if __name__=="__main__":
    main()

[{'Instant_food_products': 0.008032536858159633}, {'UHT_milk': 0.03345195729537367}, {'baking_powder': 0.017691916624300967}, {'beef': 0.05246568378240976}, {'berries': 0.033248601931875954}, {'beverages': 0.026029486527707167}, {'bottled_beer': 0.08052872394509406}, {'bottled_water': 0.11052364006100661}, {'brown_bread': 0.06487036095577021}, {'butter': 0.05541433655312659}, {'butter_milk': 0.027961362480935434}, {'cake_bar': 0.013218098627351297}, {'candles': 0.008947635993899338}, {'candy': 0.0298932384341637}, {'canned_beer': 0.07768174885612608}, {'canned_fish': 0.015048296898830707}, {'canned_vegetables': 0.010777834265378749}, {'cat_food': 0.023284189120488054}, {'cereals': 0.0056939501779359435}, {'chewing_gum': 0.021047280122013217}, {'chicken': 0.04290798169801729}, {'chocolate': 0.04961870869344179}, {'chocolate_marshmallow': 0.009049313675648195}, {'citrus_fruit': 0.08276563294356888}, {'cleaner': 0.005083884087442806}, {'cling_film_bags': 0.011387900355871887}, {'coffee': 